In [5]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from dt_apriltags import Detector
import matplotlib.cm as cm
from lane_detection import *
from lane_following import *

In [ ]:
def drawLines(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 90, 100, apertureSize=3) 
    lines = cv2.HoughLinesP(
                    edges, #described above
                    1, #1 pixel resolution parameter
                    np.pi/180, # 1 degree resolution parameter
                    10, #min number of intersections/votes
                    minLineLength=10,
                    maxLineGap=60,
            ) # detect lines
    try:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            slope = (y2-y1)/(x2-x1)
            print(str(slope))
    except TypeError:
        pass
    
    return img


In [ ]:
img = cv2.imread('rov_pool.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
edges = cv2.Canny(gray, 106, 107, apertureSize=3) # detect edges, gray is image in grayscale, 50 and 150 represent 2 images that have been threshholded at 2 different levels, apertureSize controls how much light the image gets and how exposed it is
lines = cv2.HoughLinesP(
                edges, #described above
                1, #1 pixel resolution parameter
                np.pi/180, # 1 degree resolution parameter
                60, #min number of intersections/votes
                minLineLength=475,
                maxLineGap=100,
        ) # detect lines

for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    slope = (y2-y1)/(x2-x1)
    print(str(slope))

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
import cv2
import numpy as np

video = cv2.VideoCapture('AUV_Vid.mkv')

fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_file = 'output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_file, fourcc, 30, (width, height))

ret, frame = video.read()
count = 0
frequency = 1

while ret:
    if count % frequency == 0:
        processed_frame = drawLines(frame)
        output_video.write(processed_frame)

    count += 1
    ret, frame = video.read()

video.release()
output_video.release()

In [ ]:
from dt_apriltags import Detector
import cv2
import numpy as np


cameraMatrix = np.array([ 1060.71, 0, 960, 0, 1060.71, 540, 0, 0, 1]).reshape((3,3))

camera_params = ( cameraMatrix[0,0], cameraMatrix[1,1], cameraMatrix[0,2], cameraMatrix[1,2] )


video = cv2.VideoCapture('AprilTagTest.mkv')

fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_file = 'output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_file, fourcc, 30, (width, height))

ret, frame = video.read()
count = 0
frequency = 1


while ret:
    if count % frequency == 0:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        at_detector = Detector(families='tag36h11', #controls what it is supposed to detect
                       nthreads=1, #controls the number of threads used in the detection
                       quad_decimate=1.0, #controls to what scale the image is lowered in resolution
                       quad_sigma=0.0, #blur applied to the image to reduce noise
                       refine_edges=1, #attempts to redefine the edges of the tag boundaries
                       decode_sharpening=0.25, #controls how much the image is sharpened after the tags are discovered
                       debug=0) # no debugging information will be produced

        tags = at_detector.detect(frame, estimate_tag_pose=True, camera_params=camera_params, tag_size=0.1)

        color_frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)

        for tag in tags:
            for idx in range(len(tag.corners)):
                cv2.line(color_frame, tuple(tag.corners[idx - 1, :].astype(int)), tuple(tag.corners[idx, :].astype(int)), (0, 255, 0))

            cv2.putText(color_frame, str(tag.tag_id),
                        org=(tag.corners[0, 0].astype(int) + 10, tag.corners[0, 1].astype(int) + 10),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=0.8,
                        color=(0, 0, 255))
            print(tag.pose_t)
            print(tag.pose_R)

        output_video.write(color_frame)

    count += 1
    ret, frame = video.read()

video.release()
output_video.release()

In [6]:
cap = cv2.VideoCapture('AUV_Vid.mkv')
img_array = []
i = 0
ret = True
detectedLane = 0
failedToDetectLane = 0
while ret:
    if i > 1300:
        break
    slopeSet = set() 
    i += 1
    ret, frame = cap.read()
    if(i%1 == 0 and i > 800):
        lines = detect_lines(frame, 30, 90, 3,10,10)
    
        try:
            lanes = detect_lanes(lines)
            pickedLane = pick_lane(lanes)
            frame = draw_Single_lane(frame, pickedLane, (255, 0, 0))
            frame = draw_lines(frame, lines,(0, 255, 0))
            center_intercept, center_slope = get_lane_center(pickedLane)
            xPoint = pickedLane[0][2]
            yPoint = pickedLane[0][3]
            cv2.line(frame, (int(center_intercept), 1080), (int(xPoint), int(yPoint)), (0,0,255), 3)
            rec_dir = recommend_direction(center_intercept, center_slope)
            print (rec_dir)
            cv2.putText(frame, rec_dir, (0, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            detectedLane += 1
        except:
            failedToDetectLane += 1
            pass
        
        img_array.append(frame)
        height, width, layers = frame.shape
        size = (width,height)
    
        accuracy = (detectedLane/(detectedLane+failedToDetectLane))*100
        cv2.putText(frame, str(accuracy), (0, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        print(f"accuracy: {accuracy}%")
        print(f"currently on frame: {i}")

out = cv2.VideoWriter('testVid8.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

accuracy: 0.0%
currently on frame: 801
accuracy: 0.0%
currently on frame: 802
accuracy: 0.0%
currently on frame: 803
accuracy: 0.0%
currently on frame: 804
1/ slope:1.1818181818181819
Strafe Right by -988.6333333333332 turn Left by: -50.75696144098341 degrees
accuracy: 20.0%
currently on frame: 805
accuracy: 16.666666666666664%
currently on frame: 806
accuracy: 14.285714285714285%
currently on frame: 807
accuracy: 12.5%
currently on frame: 808


/home/travistran/cv-intro/lane_detection.py:84: RuntimeWarning: divide by zero encountered in scalar divide
  intercept = ((((2160 - y1)/slope)  )+ x1)


accuracy: 11.11111111111111%
currently on frame: 809
accuracy: 10.0%
currently on frame: 810
accuracy: 9.090909090909092%
currently on frame: 811
accuracy: 8.333333333333332%
currently on frame: 812
accuracy: 7.6923076923076925%
currently on frame: 813
accuracy: 7.142857142857142%
currently on frame: 814
accuracy: 6.666666666666667%
currently on frame: 815
accuracy: 6.25%
currently on frame: 816
accuracy: 5.88235294117647%
currently on frame: 817
accuracy: 5.555555555555555%
currently on frame: 818
accuracy: 5.263157894736842%
currently on frame: 819
accuracy: 5.0%
currently on frame: 820
accuracy: 4.761904761904762%
currently on frame: 821
accuracy: 4.545454545454546%
currently on frame: 822
accuracy: 4.3478260869565215%
currently on frame: 823
accuracy: 4.166666666666666%
currently on frame: 824
accuracy: 4.0%
currently on frame: 825
accuracy: 3.8461538461538463%
currently on frame: 826
accuracy: 3.7037037037037033%
currently on frame: 827
accuracy: 3.571428571428571%
currently on fr

/home/travistran/cv-intro/lane_detection.py:106: RuntimeWarning: invalid value encountered in scalar subtract
  InterceptDist = abs(xInterceptList[i]-xInterceptList[j])


accuracy: 3.0303030303030303%
currently on frame: 833
accuracy: 2.941176470588235%
currently on frame: 834
accuracy: 2.857142857142857%
currently on frame: 835
accuracy: 2.7777777777777777%
currently on frame: 836
accuracy: 2.7027027027027026%
currently on frame: 837
accuracy: 2.631578947368421%
currently on frame: 838
accuracy: 2.564102564102564%
currently on frame: 839
accuracy: 2.5%
currently on frame: 840
accuracy: 2.4390243902439024%
currently on frame: 841
accuracy: 2.380952380952381%
currently on frame: 842
accuracy: 2.3255813953488373%
currently on frame: 843
accuracy: 2.272727272727273%
currently on frame: 844
accuracy: 2.2222222222222223%
currently on frame: 845
accuracy: 2.1739130434782608%
currently on frame: 846
accuracy: 2.127659574468085%
currently on frame: 847
accuracy: 2.083333333333333%
currently on frame: 848
accuracy: 2.0408163265306123%
currently on frame: 849
accuracy: 2.0%
currently on frame: 850
accuracy: 1.9607843137254901%
currently on frame: 851
accuracy: 1.

/home/travistran/cv-intro/lane_detection.py:110: RuntimeWarning: divide by zero encountered in scalar divide
  slopeDiff = abs(1/ slopeList[i]-1 /slopeList[j])


accuracy: 22.058823529411764%
currently on frame: 1072
accuracy: 21.978021978021978%
currently on frame: 1073
accuracy: 21.897810218978105%
currently on frame: 1074
accuracy: 21.818181818181817%
currently on frame: 1075
accuracy: 21.73913043478261%
currently on frame: 1076
accuracy: 21.660649819494584%
currently on frame: 1077
accuracy: 21.58273381294964%
currently on frame: 1078
accuracy: 21.50537634408602%
currently on frame: 1079
accuracy: 21.428571428571427%
currently on frame: 1080
accuracy: 21.352313167259787%
currently on frame: 1081
accuracy: 21.27659574468085%
currently on frame: 1082
accuracy: 21.20141342756184%
currently on frame: 1083
accuracy: 21.12676056338028%
currently on frame: 1084
accuracy: 21.052631578947366%
currently on frame: 1085
accuracy: 20.97902097902098%
currently on frame: 1086
accuracy: 20.905923344947734%
currently on frame: 1087
accuracy: 20.833333333333336%
currently on frame: 1088
accuracy: 20.761245674740483%
currently on frame: 1089
accuracy: 20.6896